In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler as sc
returnsant = pd.read_csv('returnsant.csv')

In [2]:
def encoderr(see):
     if see ==9:
        return keras.Sequential([tf.keras.layers.Dense(6,kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=see),
                                                 trainable=True),
                           tf.keras.layers.Dense(4
                           ,kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=see+1),trainable=True),
                           tf.keras.layers.Dense(2
                           ,kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=see+2),trainable=True),
                           tf.keras.layers.Dense(1,activation="relu"
                           ,kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=see+2),trainable=True)])
     else:
        return keras.Sequential([tf.keras.layers.Dense(6
                                                 ,kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=see),trainable=False),
                           tf.keras.layers.Dense(4
                           ,kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=see+1),trainable=False),
                           tf.keras.layers.Dense(2
                           ,kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=see+2),trainable=False),
                           tf.keras.layers.Dense(1,activation="relu"
                           ,kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=see+2),trainable=False)],) # extra comma for tuple
decoderr= keras.Sequential([tf.keras.layers.Dense(1),
                            tf.keras.layers.Dense(2),
                             tf.keras.layers.Dense(4),
                           tf.keras.layers.Dense(6,activation="relu")],)
encoderrs=[]
for i in range(0,10):
    encoderrs.append(encoderr(i))
encoderrs=np.array(encoderrs)

In [3]:
encoderrs

array([<tensorflow.python.keras.engine.sequential.Sequential object at 0x7fa7c469c510>,
      dtype=object)

# create error vector to weight each DATA set fed into each respective encoderr

In [28]:
class BoostedAE(keras.Model):
    def __init__(self, encoder,decoder):
        super(BoostedAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def compile(self):
        super(BoostedAE, self).compile()


    def fit(self,x,y,batch_size,epochs):
        
        tf.keras.backend.set_floatx('float64')
        loss_fn=keras.losses.MeanSquaredError()
        optimizer=keras.optimizers.Adam(learning_rate=0.00001)
        
        train_acc_metric=keras.metrics.MeanSquaredError()
        train_dataset = tf.data.Dataset.from_tensor_slices((x,y))
        train_dataset = train_dataset.batch(batch_size)
        
        for epoch in range(epochs):
            print("\nStart of epoch %d" % (epoch,))
            start_time = time.time()
            
    # Iterate over the batches of the dataset.
            
            #intialize with equal voting 1/N wewights 
            ws=[0.1]*10
            for step, (x_batch_train,y_batch_train ) in enumerate(train_dataset):
                boost=[]
                
                #print(x_batch_train)        
                
                with tf.GradientTape() as tape:
                    
                    for e in range(0,10):
                        #print(ws[e])
                        ok = self.encoder[e](ws[e]*x_batch_train)
                        ok=(tf.reshape(ok,(x_batch_train.shape[0],1)))
                        ok=tf.cast(ok,tf.float64)
                        boost.append(ok)
                        ws[e]=(np.sum(ok-y_batch_train))**2
                        #print(ws[e])
                      # print(ok)`
                        
                       # print(y_batch_train)
                        
                        #print(ok)
                        
                       # print((ok-y_batch_train)**2)
                    
                        # assert that the ws sum to 1 and hence wieghted by sum
                    if np.sum(ws)!=1:    
                        ws=np.array(ws)
                        ws=ws/np.sum(ws)
                    else:
                        pass
                    #print(ws)
                    #print(np.sum(ws))
                    #print(ws)
                   # print('boost{}'.format(boost.shape()))
                    #implement cross val aspect of training loop
                    boost=np.array(boost)
                    boost.shape = ((x_batch_train.shape[0],10))
                    # insert weighted function  on each x_i
                    
                    newx=np.average(boost,axis=1)
                    newx.shape=((x_batch_train.shape[0],1))
                    
                    predictions = self.decoder(newx)
                    
                    loss_value = loss_fn(y_batch_train, predictions)
                    loss_value += sum(self.losses)
                    
                    if step % 20 == 0:
                    
                        print(
                        "Training loss (for one batch) at step %d: %.4f"
                        % (step, float(loss_value))
                         ) 
                    
                        print("Seen so far: %d samples" % ((step + 1) * batch_size))
                    
                grads = tape.gradient(loss_value, self.trainable_weights )
                

                optimizer.apply_gradients(zip(grads, self.trainable_weights))
  
                train_acc_metric.update_state(y_batch_train,predictions)
            
            train_acc=train_acc_metric.result()
            print("Training acc over epoch: %.4f" % (float(train_acc),))
            train_acc_metric.reset_states()
            print("Time taken: %.2fs" % (time.time() - start_time))

        return {'metric_a':loss_value}
                        

In [29]:
bae=BoostedAE(encoderrs,decoderr)

In [30]:
bae.compile()

In [31]:
bae.fit(returnsant['Hedge Fund'].values.reshape(-1,1),returnsant['Hedge Fund'].values.reshape(-1,1),batch_size=20,epochs=100)


Start of epoch 0
Training loss (for one batch) at step 0: 0.0005
Seen so far: 20 samples
Training acc over epoch: 0.0009
Time taken: 2.74s

Start of epoch 1
Training loss (for one batch) at step 0: 0.0005
Seen so far: 20 samples
Training acc over epoch: 0.0009
Time taken: 2.28s

Start of epoch 2
Training loss (for one batch) at step 0: 0.0005
Seen so far: 20 samples
Training acc over epoch: 0.0009
Time taken: 2.17s

Start of epoch 3
Training loss (for one batch) at step 0: 0.0005
Seen so far: 20 samples
Training acc over epoch: 0.0009
Time taken: 2.13s

Start of epoch 4
Training loss (for one batch) at step 0: 0.0005
Seen so far: 20 samples
Training acc over epoch: 0.0009
Time taken: 2.21s

Start of epoch 5
Training loss (for one batch) at step 0: 0.0005
Seen so far: 20 samples
Training acc over epoch: 0.0009
Time taken: 2.26s

Start of epoch 6
Training loss (for one batch) at step 0: 0.0005
Seen so far: 20 samples
Training acc over epoch: 0.0009
Time taken: 2.32s

Start of epoch 7
Tr

{'metric_a': <tf.Tensor: id=394218, shape=(), dtype=float32, numpy=0.0020153874>}